In [928]:
!pip install simpy

In [929]:
import simpy
import random
import numpy as np
from random import randint

In [930]:
TOTAL_SIM = 100  # number of simulations
MINUTES_IN_DAY = 1440 # 1 day -> 1440 MIN
ARRIVAL_RATE = 2

In [931]:
class xorshift:
    max64bit = (2**64) 
 
    def __init__(self,seed):
        self.seed = seed % self.max64bit

    def rand(self):
        # a=13 b=7 c=17
        # Python no tiene limite de valores para int asi que aplico modulo de 64 bits
        # para simular el comportamiento de C y tener periodo de (2^64 - 1) ----> Confirmado: Hay que imitar la implementacion del paper
        self.seed = (self.seed ^ (self.seed << 13)) % self.max64bit
        self.seed = (self.seed ^ (self.seed >> 7)) % self.max64bit
        self.seed = (self.seed ^ (self.seed << 17)) % self.max64bit

        return self.seed
    # Dividimos por el módulo para obtener valores en [0,1]
    def uniform_rand(self):
        return self.rand()/self.max64bit

    def uniform_rand_seconds_in_day(self):
        return int(self.uniform_rand() * MINUTES_IN_DAY)

In [932]:
rng = xorshift(int(time.time()))

NameError: name 'time' is not defined

In [ ]:
class RandOp:
    
    # m:modulus
    # a:multiplier
    # c:increment
    m = 2**32
    a = 1013904223
    c = 1664525
    xn = int((97576+93956+104093+96444+98408)/5)
    
    def rand(self):
        self.xn = (self.a*self.xn + self.c)%self.m
        return self.xn
    
    # Dividimos por el módulo para obtener valores en [0,1]
    def uniform_rand(self):
        return self.rand()/self.m
    
    def uniform_rand_seconds_in_day(self):
        return int (self.uniform_rand() * MINUTES_IN_DAY)

In [ ]:
randOp= RandOp()

In [ ]:
def python_random():
    return randint(0,MINUTES_IN_DAY)

In [ ]:
class Stats:
    def __init__(self):
        self.maxRequestsWaiting = 0
        self.maxTimeWaiting = 0
        self.requestCounter = 0

In [ ]:
class Request:
    def __init__(self, env,number, random_func):
        self.number = number
        self.env = env
        self.rand_op=random_func
    def process(self):
        yield self.env.timeout(self.rand_op()) #lo que tardara en ser procesada

In [ ]:
def call(env,request,satellite, stats):
    arrive = env.now
    #print('Request %d arrives at the Satellite at minute %.2f.' % (request.number, arrive))
    with satellite.request() as req:
        yield req ##espero a que el satelite este libre para procesar la request
        wait = env.now - arrive

        if wait > stats.maxTimeWaiting:
            stats.maxTimeWaiting = wait

        #print('Request %d starts being processed at minute %.2f.' % (request.number,env.now))
        yield env.process(request.process())
        #print('Request %d leaves Satellite at minute %.2f.' % (request.number,env.now))

In [ ]:
def source(env, satellite, stats, arrival_rate,random_func):
    while True:

        #random.expovariate(15) generates exponentially distributed random numbers in a poisson process with 
        #an averate rate of 15 arrivals per second. Generates the *inter*arrival times
        t = random.expovariate(arrival_rate/60) #
        yield env.timeout(t) ##simulo llegada de req
        
        request = Request(env,stats.requestCounter,random_func)
        stats.requestCounter += 1

        queue = len(satellite.queue) ##si hay requests esperando
        if queue > stats.maxRequestsWaiting:
            stats.maxRequestsWaiting = queue

        env.process(call(env, request, satellite, stats))##call a ejecutar la request

In [ ]:
def run_sim(random_func):
    averageMaxRequestsWaiting = []
    averageMaxTimeWaiting = []
    for i in range(TOTAL_SIM):
        env = simpy.Environment()
        stats = Stats()
        satellite = simpy.Resource(env, capacity=1)
        
        env.process(source(env, satellite, stats,ARRIVAL_RATE,random_func))
        env.run(until=MINUTES_IN_DAY)
        
        averageMaxRequestsWaiting.append(stats.maxRequestsWaiting)
        averageMaxTimeWaiting.append(stats.maxTimeWaiting)
    print('*********************************************************')
    print('Cantidad de simulaciones: %d' % TOTAL_SIM)
    #print('Cantidad max de req esperando: ', *averageMaxRequestsWaiting, sep = ", ")
    #print('Tiempos maximos de espera: ', *averageMaxTimeWaiting, sep = ", ")
    print('La cantidad máxima promedio de requests en fila esperando a ser procesadas es de %d' % np.mean(averageMaxRequestsWaiting))
    print('El tiempo máximo promedio de espera en fila es de %d min' % np.mean(averageMaxTimeWaiting))

In [ ]:
run_sim(randOp.uniform_rand_seconds_in_day)